In [1]:
import json
import os
import random
import together
from datasets import load_dataset

WANDB_API_KEY= "50a4ae3d3476d2fa831569e4d22bfec32200130f"


# WANDB_API_KEY = None # replace None with your weights and Biases API key (optional)


# lets use our base model to see how it works before we finetune it

# base_model_name = "togethercomputer/Llama-2-7B-32K-Instruct"
base_model_name = "togethercomputer/LLaMA-2-7B-32K"
#base_model_name = "togethercomputer/llama-2-7b-chat"

# Specify path to your JSON file
json_path = os.path.join(os.getcwd(), 'google_translated - Copy.json')

# Open the JSON file and load it into a dict
with open(json_path, 'r') as json_file:
    ourdataset = json.load(json_file)

C:\Users\msamwelmollel\anaconda3\envs\python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def format_to_llama2_chat(system_prompt, user_model_chat_list):

    """ this function follows from
    https://docs.together.ai/docs/fine-tuning-task-specific-sequences

    It converts this ourdataset into the Llama-2 prompting structure

    Args:
      system_prompt (str): instructions from you the developer to the AI
      user_model_chat_list (List[Tuple[str,str]]): a list of tuples,
        where each tuple is a pair or exchange of string utterances, the first by the user,
        the second by the AI. The earlier exchanges are on the left, meaning time
        runs left to right.
    Returns:
      growing_prompt (str): the concatenated sequence starting with system_prompt and
        alternating utterances between the user and AI with the last AI utternance on the right.
    """

    growing_prompt = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>>"""

    for user_msg, model_answer in user_model_chat_list:
        growing_prompt += f""" {user_msg} [/INST] {model_answer} </s>"""

    return growing_prompt

format_to_llama2_chat(
    "You are a good robot",
    [("hi robot", "hello human"),("are you good?", "yes im good"),("are you bad?", "no, im good")]
)

'<s>[INST] <<SYS>> You are a good robot <</SYS>> hi robot [/INST] hello human </s> are you good? [/INST] yes im good </s> are you bad? [/INST] no, im good </s>'

In [3]:
test_chat_prompt = "<s>[INST] <<SYS>> you are a helpful  assistant <</SYS>> Je tatu jumlisha tatu ni ngapi? [/INST]"
test_chat_prompt

'<s>[INST] <<SYS>> you are a helpful  assistant <</SYS>> Je tatu jumlisha tatu ni ngapi? [/INST]'

In [4]:
data_list = []

for sample in ourdataset[0:10000]:

    instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
    # instruction_input_separator = random.choice([":" ])
    input = sample['input'] if sample['input'] is not None else ""
    instruction = sample['instruction'] if sample['instruction'] is not None else ""

    training_sequence = format_to_llama2_chat(
        "you are a helpful assistant",
        [(instruction+instruction_input_separator+input,sample['output'])]
    )

    data_list.append({
        "text":training_sequence
    })

In [5]:
# save the reformatted dataset locally
together.Files.save_jsonl(data_list, "Alpaca_Swahili_Dataset.jsonl")


# check your data with your base model prompting type before uploading
resp = together.Files.check(file="Alpaca_Swahili_Dataset.jsonl")
print(resp)


# upload your dataset file to together and save the file-id, youll need it to start your finetuning run
file_resp = together.Files.upload(file="Alpaca_Swahili_Dataset.jsonl")
file_id = file_resp["id"]
print("-"*50)
print(file_resp)

Wrote 10000 records to Alpaca_Swahili_Dataset.jsonl
{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.004 GB', 'num_samples': 10000}


Uploading Alpaca_Swahili_Dataset.jsonl: 100%|██████████| 4.15M/4.15M [00:17<00:00, 251kB/s] 

--------------------------------------------------
{'filename': 'Alpaca_Swahili_Dataset.jsonl', 'id': 'file-9ff39f85-8b9c-46b7-89f9-1c3ea15db883', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.004 GB', 'num_samples': 10000}}


In [6]:
# Submit your finetune job
ft_resp = together.Finetune.create(
  training_file = file_id ,
  model = base_model_name,
  n_epochs = 2,
  batch_size = 4,
  n_checkpoints = 1,
  learning_rate = 5e-5,
  wandb_api_key = WANDB_API_KEY,
  #estimate_price = True,
  suffix = 'swahili',
)

fine_tune_id = ft_resp['id']
print(ft_resp)

{'training_file': 'file-9ff39f85-8b9c-46b7-89f9-1c3ea15db883', 'validation_file': '', 'model_output_name': 'msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14', 'model_output_path': 's3://together-dev/finetune/65a91a45525ee5acd30edf0f/msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14/ft-7e2226aa-901c-4f47-b152-5152f4a58a74', 'Suffix': 'swahili', 'model': 'togethercomputer/LLaMA-2-7B-32K', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 5e-05, 'user_id': '65a91a45525ee5acd30edf0f', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-01-29T10:15:14.756Z', 'updated_at': '2024-01-29T10:15:14.756Z', 'status': 'pending', 'owner_address': '0x2db181983c47b0d9923e38a09f76d50d3b4596a7', 'id': 'ft-7e2226aa-901c-4f47-b152-5152f4a58a74', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'ev

In [12]:
# run this from time to time to check on the status of your job
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

{'training_file': 'file-9ff39f85-8b9c-46b7-89f9-1c3ea15db883', 'validation_file': '', 'model_output_name': 'msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14', 'model_output_path': 's3://together-dev/finetune/65a91a45525ee5acd30edf0f/msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14/ft-7e2226aa-901c-4f47-b152-5152f4a58a74', 'Suffix': 'swahili', 'model': 'togethercomputer/LLaMA-2-7B-32K', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 5e-05, 'user_id': '65a91a45525ee5acd30edf0f', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-01-29T10:15:14.756Z', 'updated_at': '2024-01-29T10:17:27.287Z', 'status': 'running', 'owner_address': '0x2db181983c47b0d9923e38a09f76d50d3b4596a7', 'id': 'ft-7e2226aa-901c-4f47-b152-5152f4a58a74', 'job_id': '11977', 'token_count': 1740606, 'param_count': 6738415616, 'total_price': 500000000

In [14]:
# replace this name with the name of your newly finetuned model
new_model_name = 'msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14'
# replace this name with the name of your newly finetuned model
# new_model_name = 'togethercomputer/llama-2-7b-chat'




<s> [INST] <<SYS>> you are a helpful assistant <</SYS>> Pretend you are a farmer. [/INST]
model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

129 models available


True

In [15]:
# deploy your newly finetuned model
together.Models.start(new_model_name)

{'success': True,
 'value': '163d4b015ade343d869897c454bf68b9faae962ddc10bc6d88ed998281031bb7-ae0d202637b02617e660c3565b39d2e145465efa352c37238da32e0361922540'}

In [16]:
# check if your model is finished deploying, if this returns {"ready": true}, you model is ready for inference
together.Models.ready(new_model_name)

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0},
  '_id': '65b77f5313e58adb27374e47',
  'name': 'msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14',
  'display_type': 'language',
  'description': "Extending LLaMA-2 to 32K context, built with Meta's Position Interpolation and Together AI's data recipe and system optimizations.",
  'creator_organization': 'msamwelmollel@gmail.com',
  'hardware_label': 'L40',
  'num_parameters': '6738415616',
  'release_date': '2024-01-29T10:34:59.193Z',
  'show_in_playground': True,
  'owner': 'msamwelmollel@gmail.com',
  'owner_address': '0x2db181983c47b0d9923e38a09f76d50d3b4596a7',
  'owner_userid': '65a91a45525ee5acd30edf0f',
  'parent': 'togethercomputer/LLaMA-2-7B-32K',
  'base': 'togethercomputer/LLaMA-2-7B-32K',
  'path': 'r2://together-dev/finetune/65a91a45525ee5acd30edf0f/msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14/ft-7e2226aa-901c-4f47-b152-5152f4a58a74-2024-01-29-02-29-58',
  'files': [{'filename'

In [17]:
test_chat_prompt = "<s>[INST] <<SYS>> you are a helpful  assistant <</SYS>> Toa vidokezo vitatu vya kudumisha afya. [/INST]"
test_chat_prompt

'<s>[INST] <<SYS>> you are a helpful  assistant <</SYS>> Toa vidokezo vitatu vya kudumisha afya. [/INST]'

In [21]:
TOGETHER_API_KEY = "28404b202e87f83fa2534b925cd9382f02a6d3780b5fba632524351ff98d851f"

In [ ]:
# use the inference API to generate text / create completion / chat
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = new_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14


In [19]:
# use the inference API to generate text / create completion / chat
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = new_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

msamwelmollel@gmail.com/LLaMA-2-7B-32K-swahili-2024-01-29-10-15-14


HTTPError: 404 Client Error: Not Found for url: https://api.together.xyz/api/inference

In [20]:
# stop your model and you will no longer be paying for it
together.Models.stop(new_model_name)

{'success': True}

In [ ]:
import json
import os

import requests
import sseclient

url = "https://api.together.xyz/inference"
model = "togethercomputer/RedPajama-INCITE-7B-Chat"
prompt = "Tell me a story\n\n"

print(f"Model: {model}")
print(f"Prompt: {repr(prompt)}")
print("Repsonse:")
print()

payload = {
    "model": model,
    "prompt": prompt,
    "max_tokens": 512,
    "temperature": 0.7,
    "top_p": 0.7,
    "top_k": 50,
    "repetition_penalty": 1,
    "stream_tokens": True,
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}",
}

response = requests.post(url, json=payload, headers=headers, stream=True)
response.raise_for_status()

client = sseclient.SSEClient(response)
for event in client.events():
    if event.data == "[DONE]":
        break

    partial_result = json.loads(event.data)
    token = partial_result["choices"][0]["text"]
    print(token, end="", flush=True)     